<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-19 11:13:36
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.43 C
-------------------
Today PnL: -66.24 K (-0.46%)
Current PnL: -31.84 L (-19.9%)
CY Booked + Current PnL: -16.16 L (-10.1%)
-------------------
Total profit:  1.31 L
Total loss:  -33.14 L
-------------------
Total Booked + Current PnL: 10.82 L (8.14%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.0%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 98.45 L (69.04%)
Deployed:  1.33 C
Current:  1.43 C
CAGR/XIRR %: 3.61%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,0.18,3.37,17.61,21.57,17835.0,3300.0,101280.0,397.04,-11.02,60.0,H-SC,5.16,167.0,0.19,0.72,47.80,XR,ATH,FINANCE
18,COALINDIA,-0.17,2.62,14.67,17.67,22651.0,3940.0,154401.0,478.51,23.39,50.0,L-LC,11.48,185.0,0.17,1.10,23.95,XY25,ATH,MINING
77,TTKPRESTIG,-0.55,-28.19,39.39,0.09,28504.0,-28413.0,72364.0,770.00,67.98,37.0,M-SC,9.45,250.0,-1.00,0.52,0.00,OX40N,NTT,DURABLES
36,ICICIGI,-0.46,3.33,15.50,19.34,32048.0,6655.0,206764.0,2252.93,-12.97,62.0,X-MC,2.57,66.0,0.21,1.48,19.79,X40,ATH,INSURANCE
51,MEDANTA,-1.96,-3.97,29.21,24.08,35610.0,-5039.0,121911.0,1486.00,-4.57,53.0,X-SC,10.61,87.0,-0.14,0.87,11.66,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BSOFT,3.05,-29.79,114.77,50.79,113334.0,-41904.0,98749.0,831.70,-9.93,40.0,H-SC,6.78,172.0,-0.37,0.70,14.98,XR,ATH,IT
49,LAOPALA,2.04,-39.84,131.20,39.09,103479.0,-52234.0,78871.0,464.00,72.77,53.0,H-SC,14.74,154.0,-0.50,0.56,5.47,AR,NTT,CERAMICS
43,IRCTC,1.64,-23.63,100.25,52.93,207102.0,-63922.0,206586.0,1269.00,-43.47,52.0,H-MC,12.39,119.0,-0.31,1.47,4.31,XY24,BTT,TRAVEL
42,INFY,1.45,-5.13,41.50,34.24,123187.0,-16051.0,296837.0,1972.00,-25.25,29.0,X-LC,3.88,3.0,-0.13,2.12,2.73,X40,NTT,IT
30,HAPPSTMNDS,1.16,-51.18,278.65,84.85,174346.0,-65597.0,62568.0,1480.71,-41.98,41.0,H-SC,25.67,152.0,-0.38,0.45,5.15,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,DIXON,-2.80,-22.39,69.12,31.26,123797.0,-51664.0,179104.0,18931.72,-65.30,46.0,X-MC,25.31,54.0,-0.42,1.28,9.92,X40N,ATH,IT
7,ATULAUTO,-2.43,-12.36,66.09,45.57,114184.0,-24361.0,172771.0,844.00,3748.65,64.0,M-SC,4.07,248.0,-0.21,1.23,30.75,XY24,NTT,AUTO
25,EASEMYTRIP,-2.28,-48.07,180.55,45.70,165786.0,-84992.0,91823.0,26.40,-4.69,77.0,M-SC,21.29,204.0,-0.51,0.66,51.29,XY24,NTT,TRAVEL
19,COFFEEDAY,-2.15,-45.28,147.22,35.27,91469.0,-51418.0,62131.0,80.00,-57.75,49.0,L-SC,14.39,266.0,-0.56,0.44,52.07,XR,NTT,HOTELS
58,RELAXO,-2.12,-52.76,222.41,52.31,152513.0,-76587.0,68573.0,1176.00,-49.67,43.0,X-SC,13.98,92.0,-0.50,0.49,2.80,X40N,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,-0.72,3.10,101.66,107.91,154745.0,4574.0,152218.0,1641.55,-7.15,63.0,M-SC,8.51,220.0,0.03,1.09,14.52,OX40N,ATH,CABLES
77,TTKPRESTIG,-0.55,-28.19,39.39,0.09,28504.0,-28413.0,72364.0,770.00,67.98,37.0,M-SC,9.45,250.0,-1.00,0.52,0.00,OX40N,NTT,DURABLES
72,TATAELXSI,-0.69,-29.60,87.53,32.03,81241.0,-39020.0,92815.0,9161.00,-11.96,36.0,H-SC,7.23,158.0,-0.48,0.66,3.96,OX40N,NTT,IT
66,SIS,-1.58,-29.69,72.46,21.26,56568.0,-32964.0,78068.0,528.00,1834.72,34.0,H-SC,4.32,168.0,-0.58,0.56,7.62,OX40N,NTT,MISC
40,INDIGOPNTS,0.12,-30.69,44.31,0.02,53609.0,-53572.0,120987.0,1408.00,84.71,31.0,M-SC,9.21,222.0,-1.00,0.86,5.24,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,0.01,20.75,20.25,45.2,46884.0,39783.0,231525.0,1856.0,38.21,68.0,X-MC,5.28,79.0,0.85,1.65,28.96,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.17,2.62,14.67,17.67,22651.0,3940.0,154401.0,478.51,23.39,50.0,L-LC,11.48,185.0,0.17,1.10,23.95,XY25,ATH,MINING
26,FINCABLES,-0.72,3.10,101.66,107.91,154745.0,4574.0,152218.0,1641.55,-7.15,63.0,M-SC,8.51,220.0,0.03,1.09,14.52,OX40N,ATH,CABLES
50,MASFIN,0.18,3.37,17.61,21.57,17835.0,3300.0,101280.0,397.04,-11.02,60.0,H-SC,5.16,167.0,0.19,0.72,47.80,XR,ATH,FINANCE
1,ABB,-0.22,7.67,35.20,45.57,99154.0,20069.0,281688.0,7934.00,-30.81,68.0,H-MC,5.12,125.0,0.20,2.01,25.09,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-0.22,7.67,35.20,45.57,99154.0,20069.0,281688.0,7934.00,-30.81,68.0,H-MC,5.12,125.0,0.20,2.01,25.09,AR,NTT,ELECTRICAL
50,MASFIN,0.18,3.37,17.61,21.57,17835.0,3300.0,101280.0,397.04,-11.02,60.0,H-SC,5.16,167.0,0.19,0.72,47.80,XR,ATH,FINANCE
26,FINCABLES,-0.72,3.10,101.66,107.91,154745.0,4574.0,152218.0,1641.55,-7.15,63.0,M-SC,8.51,220.0,0.03,1.09,14.52,OX40N,ATH,CABLES
39,INDIAMART,-0.24,-6.57,121.26,106.72,139732.0,-8103.0,115233.0,4810.62,-57.81,43.0,H-SC,7.87,140.0,-0.06,0.82,15.27,AR,ATH,MISC
85,ZYDUSLIFE,0.21,-3.83,41.04,35.63,82712.0,-8033.0,201541.0,1286.17,-16.22,53.0,H-MC,5.22,120.0,-0.10,1.44,12.20,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,-0.69,-34.77,140.97,57.19,299902.0,-113385.0,212742.0,485.00,-62.73,28.0,X-SC,17.39,81.0,-0.38,1.52,0.00,XY24,NTT,FMCG
42,INFY,1.45,-5.13,41.50,34.24,123187.0,-16051.0,296837.0,1972.00,-25.25,29.0,X-LC,3.88,3.0,-0.13,2.12,2.73,X40,NTT,IT
33,HCLTECH,0.65,-4.16,29.21,23.83,67726.0,-10066.0,231858.0,1908.19,0.50,30.0,X-LC,4.33,8.0,-0.15,1.65,11.24,X40,ATH,IT
13,BERGEPAINT,0.05,-17.94,47.62,21.14,88841.0,-40776.0,186563.0,680.00,-19.66,30.0,X-MC,12.77,68.0,-0.46,1.33,0.47,XY24,NTT,PAINTS
73,TCS,1.05,-21.95,58.33,23.58,196959.0,-94965.0,337664.0,4311.59,-32.48,30.0,X-LC,4.76,2.0,-0.48,2.41,1.15,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,-0.56,4.06,16.05,20.77,39107.0,9511.0,243655.0,1644.00,-7.70,61.0,X-MC,1.09,64.0,0.24,1.74,11.76,X40N,NTT,BREWERIES
21,DABUR,-0.40,-0.57,44.53,43.70,109832.0,-1421.0,246647.0,735.00,-9.68,47.0,X-MC,1.81,70.0,-0.01,1.76,14.39,XY24,BTT,FMCG
65,SIEMENS,-0.21,-14.53,46.85,25.51,74517.0,-27040.0,159055.0,4671.50,33.57,56.0,H-LC,2.00,51.0,-0.36,1.13,17.21,AR,ATH,ELECTRICAL
36,ICICIGI,-0.46,3.33,15.50,19.34,32048.0,6655.0,206764.0,2252.93,-12.97,62.0,X-MC,2.57,66.0,0.21,1.48,19.79,X40,ATH,INSURANCE
34,HINDUNILVR,-0.09,-9.69,25.87,13.67,60057.0,-24913.0,232150.0,2922.00,-32.95,42.0,X-LC,2.85,9.0,-0.41,1.66,9.60,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,-0.69,-34.77,140.97,57.19,299902.0,-113385.0,212742.0,485.00,-62.73,28.0,X-SC,17.39,81.0,-0.38,1.52,0.00,XY24,NTT,FMCG
53,PGHH,-0.38,-13.45,52.53,32.01,97075.0,-28720.0,184800.0,17616.87,-41.49,34.0,X-MC,9.30,57.0,-0.30,1.32,0.00,X40,ATH,FMCG
3,ACC,-0.11,-31.13,138.35,64.15,226728.0,-74071.0,163880.0,3906.00,-48.05,39.0,X-SC,8.64,85.0,-0.33,1.17,0.40,XY24,BTT,CEMENT
13,BERGEPAINT,0.05,-17.94,47.62,21.14,88841.0,-40776.0,186563.0,680.00,-19.66,30.0,X-MC,12.77,68.0,-0.46,1.33,0.47,XY24,NTT,PAINTS
2,ABBOTINDIA,-0.02,-11.25,33.04,18.07,52444.0,-20125.0,158730.0,35195.00,-24.45,38.0,X-MC,10.66,58.0,-0.38,1.13,0.63,X40,ATH,PHARMA


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-1.46,-30.95,106.86,42.84,47967.0,-20118.0,44888.0,424.00,-54.82,48.0,X-SC,18.26,80.0,-0.42,0.32,7.98,XY24,NTT,MISC
58,RELAXO,-2.12,-52.76,222.41,52.31,152513.0,-76587.0,68573.0,1176.00,-49.67,43.0,X-SC,13.98,92.0,-0.50,0.49,2.80,X40N,NTT,FOOTWEAR
12,BATAINDIA,0.01,-46.01,150.40,35.20,104490.0,-59195.0,69475.0,2096.00,-8.88,37.0,X-SC,21.57,91.0,-0.57,0.50,0.99,X40,NTT,FOOTWEAR
51,MEDANTA,-1.96,-3.97,29.21,24.08,35610.0,-5039.0,121911.0,1486.00,-4.57,53.0,X-SC,10.61,87.0,-0.14,0.87,11.66,XY24,NTT,HEALTHCARE
35,HONAUT,0.03,-22.92,85.94,43.32,107889.0,-37332.0,125540.0,58357.33,-32.37,39.0,X-SC,11.40,90.0,-0.35,0.90,1.93,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.68,-13.03,36.89,19.06,109142.0,-44307.0,295859.0,452.00,-52.28,53.0,X-LC,18.29,1.0,-0.41,2.11,7.12,X40,NTT,FMCG
73,TCS,1.05,-21.95,58.33,23.58,196959.0,-94965.0,337664.0,4311.59,-32.48,30.0,X-LC,4.76,2.0,-0.48,2.41,1.15,X40,ATH,IT
42,INFY,1.45,-5.13,41.50,34.24,123187.0,-16051.0,296837.0,1972.00,-25.25,29.0,X-LC,3.88,3.0,-0.13,2.12,2.73,X40,NTT,IT
75,TMPV,0.30,-14.67,56.25,33.33,156384.0,-47784.0,278016.0,600.00,-81.44,65.0,X-LC,3.02,4.0,-0.31,1.98,13.66,XY24,NTT,AUTO
33,HCLTECH,0.65,-4.16,29.21,23.83,67726.0,-10066.0,231858.0,1908.19,0.50,30.0,X-LC,4.33,8.0,-0.15,1.65,11.24,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-0.58,-30.66,91.66,32.89,49067.0,-23674.0,53532.0,1800.00,-352.10,61.0,L-MC,11.19,258.0,-0.48,0.38,47.42,XR,NTT,BANKS
67,SONACOMS,0.13,-7.88,50.99,39.09,47517.0,-7971.0,93188.0,804.02,-26.95,64.0,M-MC,7.64,193.0,-0.17,0.66,31.83,AR,ATH,AUTO
50,MASFIN,0.18,3.37,17.61,21.57,17835.0,3300.0,101280.0,397.04,-11.02,60.0,H-SC,5.16,167.0,0.19,0.72,47.80,XR,ATH,FINANCE
18,COALINDIA,-0.17,2.62,14.67,17.67,22651.0,3940.0,154401.0,478.51,23.39,50.0,L-LC,11.48,185.0,0.17,1.10,23.95,XY25,ATH,MINING
7,ATULAUTO,-2.43,-12.36,66.09,45.57,114184.0,-24361.0,172771.0,844.00,3748.65,64.0,M-SC,4.07,248.0,-0.21,1.23,30.75,XY24,NTT,AUTO


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,-2.28,-48.07,180.55,45.70,165786.0,-84992.0,91823.0,26.40,-4.69,77.0,M-SC,21.29,204.0,-0.51,0.66,51.29,XY24,NTT,TRAVEL
11,BANDHANBNK,0.83,-16.70,130.99,92.42,303960.0,-46511.0,232048.0,400.00,98.99,73.0,H-SC,6.64,174.0,-0.15,1.66,30.92,XY24,NTT,BANKS
1,ABB,-0.22,7.67,35.20,45.57,99154.0,20069.0,281688.0,7934.00,-30.81,68.0,H-MC,5.12,125.0,0.20,2.01,25.09,AR,NTT,ELECTRICAL
67,SONACOMS,0.13,-7.88,50.99,39.09,47517.0,-7971.0,93188.0,804.02,-26.95,64.0,M-MC,7.64,193.0,-0.17,0.66,31.83,AR,ATH,AUTO
7,ATULAUTO,-2.43,-12.36,66.09,45.57,114184.0,-24361.0,172771.0,844.00,3748.65,64.0,M-SC,4.07,248.0,-0.21,1.23,30.75,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.76
1,25,45.47
2,50,77.30


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.36
MC    29.89
LC    24.77
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.06
X40      23.98
X40N     14.87
AR        8.87
XY25      8.72
XR        8.54
OX40N     7.13
SR        0.85
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.52
H-SC    22.23
X-LC    21.48
X-SC    11.61
M-SC    10.77
H-MC     4.92
M-MC     2.07
H-LC     1.13
L-LC     1.10
M-LC     1.06
L-SC     0.75
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.54,-13.32,51.64
FINANCE,13.11,-13.60,58.31
IT,11.72,-33.50,100.40
MISC,6.92,-34.12,88.99
ELECTRICAL,6.16,-8.66,48.31
PAINTS,4.99,-32.07,51.69
INSURANCE,4.81,-0.68,35.13
PHARMA,4.05,-3.45,36.02
AUTO,3.87,-14.73,58.47


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3334752.0,21
X40,1360770.0,15
AR,1328504.0,10
XR,1294282.0,12
X40N,1038364.0,10
OX40N,792734.0,10
XY25,395875.0,6
SR,299929.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3678574.0,24
M-SC,1408698.0,14
X-MC,1357484.0,15
X-LC,1261305.0,12
X-SC,1179418.0,10
H-MC,388968.0,3
L-SC,185543.0,2
M-LC,143941.0,1
M-MC,95044.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1246732.0      6
           AR         950475.0      5
           XR         831582.0      7
M-SC       XY24       807542.0      6
X-LC       X40        643603.0      6
X-SC       XY24       610207.0      4
X-MC       X40        541011.0      7
           X40N       419642.0      4
X-SC       X40N       393055.0      4
H-SC       OX40N      349856.0      4
M-SC       OX40N      348804.0      5
H-SC       SR         299929.0      2
X-LC       XY24       264496.0      2
           X40N       225667.0      2
H-MC       XY24       207102.0      1
X-MC       XY24       198673.0      2
           XY25       198158.0      2
H-MC       AR         181866.0      2
M-SC       XR         178223.0      2
X-SC       X40        176156.0      2
M-LC       XR         143941.0      1
X-LC       XY25       127539.0      2
L-SC       OX40N       94074.0      1
           XR          91469.0      1
H-LC       AR          74517.0      1
M-SC       AR          74129.0      1
L-MC       XR          49067.0      1
M-MC       XY25        47527.0      1
           AR          47517.0      1
L-LC       XY25        22651.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
